In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
bike2019 =pd.read_csv('../input/ford-gobike-2019feb-tripdata/201902-fordgobike-tripdata.csv')

# 1.Data extraction

In [ ]:
bike2019.head()

In [ ]:
bike2019.describe()

In [ ]:
bike2019.info()

# Data Cleaning

In [ ]:
bike2019.info()

In [ ]:
bike2019[bike2019.start_station_name.isnull()][["start_station_name","start_station_latitude","start_station_longitude","end_station_name","end_station_latitude","end_station_longitude"]]

In [ ]:
bike2019[bike2019.start_station_name.isnull()].equals(bike2019[bike2019.end_station_name.isnull()])

With every start station that's missing, the end station is also missing

# Same Longtiude and Latitude for missing Start stations and End stations so we can use google maps to findout where the location is exactly and substitute the start station name and id with that location and give a random ID -Same goes with end station and end station ID-


In [ ]:
bike2019["start_time"]= pd.to_datetime(bike2019["start_time"])
bike2019["end_time"]= pd.to_datetime(bike2019["end_time"])

In [ ]:
check = bike2019['start_time'] + pd.to_timedelta(bike2019['duration_sec'], unit='s')
check

In [ ]:
differnce = pd.concat([bike2019['end_time'],check,bike2019['end_time']-check],axis=1)
differnce.columns = ["End_time","Calculated End_time","Absoulte Difference"]
differnce

In [ ]:
from datetime import datetime, timedelta 
print("Minimum Difference:",differnce["Absoulte Difference"].min())
print("Mean Difference:",differnce["Absoulte Difference"].mean())
print("Max Difference:",differnce["Absoulte Difference"].max())

Average Difference is less than 0.5 seconds, so the columns Duration, Start_Time and End_time are very accurate

# Age and Birth Year

In [ ]:
print("Average Age:", 2021- bike2019.member_birth_year.mean())
print("Average Male Age:", 2021- bike2019[bike2019["member_gender"]=="Male"].member_birth_year.mean())
print("Average Female Age:", 2021- bike2019[bike2019["member_gender"]=="Female"].member_birth_year.mean())

Ages are very close to each other, so we could just fill the missing values with the average age

Note that: Some birthdates are of people way too old, more than 90 years! those records could either be omitted or depending if ford were collecting data from the 1900s

In [ ]:
#Fill Null values with average age
bike2019["member_birth_year"] =bike2019["member_birth_year"].fillna(round(bike2019["member_birth_year"].mean())).astype('int32')
bike2019["member_birth_year"].head()

In [ ]:
bike2019["member_gender"].value_counts(dropna=False)

# Fill Missing Gender randomly


In [ ]:
bike2019["member_gender"]= bike2019["member_gender"].fillna(np.random.choice(["Male","Female"]))

In [ ]:
bike2019.info()

# For now drop the ~200 records with missing Start/End stations

In [ ]:
bike2019.dropna().info()

In [ ]:
from sklearn import preprocessing
df = bike2019.dropna(axis=0)
df[["member_gender","bike_share_for_all_trip","user_type","start_station_name","end_station_name"]] =df[["member_gender","bike_share_for_all_trip","user_type","start_station_name","end_station_name"]].apply(preprocessing.LabelEncoder().fit_transform)
df

In [ ]:
import seaborn as sns
%matplotlib inline


# calculate the correlation matrix
corr = df.corr()

# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '15pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

# As we can see from the Correlation Matrix,
1. The bikeshare is correlated with the start/end station,meaning some areas are more likely to have more bike shares than others
2. Gender is pretty much uncorrelated with anything else
3. User type (Customer/Subscriber) is marginally correlated with the duration of the trip